In [4]:
from pyomo.environ import * 
from pyomo.opt import SolverFactory

SEED = 7
m = ConcreteModel()

In [ ]:
## Generator

In [3]:
m.I = Set()
m.J = Set()
m.K = Set()

In [ ]:
m.x1 = Var(m.I, m.K, domain = NonNegativeReals)
m.x2 = Var(m.K, m.J, domain = NonNegativeReals)
m.y  = Var(m.K, domain = Boolean)

In [ ]:
m.a  = Param(m.J, domain = NonNegativeReals)
m.b  = Param(m.J, domain = NonNegativeReals)
m.c1 = Param(m.I, m.K, domain = NonNegativeReals)
m.c2 = Param(m.K, m.J, domain = NonNegativeReals)
m.f  = Param(m.K, domain = NonNegativeReals)
m.q  = Param(m.K, domain = NonNegativeReals)

In [ ]:
m.constraints=ConstraintList()

def source_out(m,i):
    return sum(m.x[i,k] for k in m.K) == m.a[i]
m.constraints.add(m.I, rule = source_out)

def process_in(m,k):
    return sum(m.x[i,k] for i in m.I) == m.q[k]
m.constraints.add(m.K, rule = process_in)

def process_out(m,j):
    return sum(m.x[k,j] for k in m.K) == m.b[j]
m.constraints.add(m.J, rule = process_out)

In [ ]:
def obj_expression():
    sum = 0
    for k in m.K:
        sum += m.y[k]*m.f[k]
        for i in m.I:
            sum += m.x1[i,k]*m.c1[i,k]
        for j in m.J:
            sum += m.x2[k,j]*m.c2[k,j]
    return sum

m.objective = Objective(rule = obj_expression, sense=minimize)

In [ ]:
solver = SolverFactory('glpk')
sol = solver.solve(m, report_timing=True, tee=True)

In [ ]:
print("The solver terminated with the following decision variable values:")              
for j in model.j:
    if(value(model.x[j])>0):
       print("  ", j, format(value(model.x[j]),".2f"))